In [ ]:
# Importing required libraries
import pandas as pd # Data manipulation library
import random 

# Define a pool of 30 unique supermarket items
item_pool = [ # List of items available in the supermarket
    'Milk', 'Bread', 'Eggs', 'Butter', 'Cheese', 'Apples', 'Bananas', 'Chicken',
    'Beef', 'Fish', 'Yogurt', 'Juice', 'Cereal', 'Rice', 'Pasta', 'Tomatoes',
    'Potatoes', 'Onions', 'Lettuce', 'Carrots', 'Beans', 'Peas', 'Toilet Paper',
    'Soap', 'Shampoo', 'Toothpaste', 'Water', 'Soda', 'Coffee', 'Tea'
] # A list of items to simulate transactions

# Set a random seed for reproducibility
random.seed(42)

# Generate 3000 transactions with 2–7 random items each
transactions = []
for _ in range(3000):
    num_items = random.randint(2, 7)
    transaction = random.sample(item_pool, num_items)
    transactions.append(', '.join(transaction))  # Join items into a comma-separated string

# Save transactions to CSV
df_transactions = pd.DataFrame({'Transaction': transactions})
df_transactions.to_csv('supermarket_transactions.csv', index=False)

# Preview first 5 transactions
print(df_transactions.head())


                                         Transaction
0  Butter, Milk, Soap, Beef, Chicken, Toothpaste,...
1    Butter, Peas, Soap, Onions, Eggs, Lettuce, Rice
2                                         Milk, Eggs
3                         Chicken, Potatoes, Carrots
4                                    Onions, Bananas
